In [ ]:
import pandas as pd

sentences_val = pd.read_csv("sentences_test.csv", sep = '\t')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold

# Parameters
folds = 5
fold_size = 150
total_rows = folds * fold_size

# # Create an example dataframe with 100 sentences
# np.random.seed(42)  # For reproducibility
# labels = np.random.choice([np.nan, 0, 1, 2, 3, 4, 5], size=100, p=[0.6, 0.19, 0.09, 0.04, 0.04, 0.02, 0.02])
# sentences = [f"Sentence {i+1}" for i in range(100)]

# # Construct the dataframe
# sentences_val = pd.DataFrame({
#     "sentence": sentences,
#     "manual_sentence_labels": labels
# })

# Reduce to 60 rows, removing only np.nan values
nan_rows = sentences_val[sentences_val['manual_sentence_labels'].isna()]
int_rows = sentences_val[~sentences_val['manual_sentence_labels'].isna()]

# Calculate the number of rows to remove
rows_to_remove = len(sentences_val) - total_rows

# Randomly remove rows from the nan_rows
rows_to_drop = nan_rows.sample(n=rows_to_remove, random_state=42)
sentences_val_reduced = sentences_val.drop(rows_to_drop.index)

# Replace np.nan with a placeholder for stratification
sentences_val_reduced['manual_sentence_labels'] = sentences_val_reduced['manual_sentence_labels'].fillna(-1)

# StratifiedKFold to create stratified samples
skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)
stratified_samples = []

# Generate the stratified samples
for _, sample_index in skf.split(sentences_val_reduced, sentences_val_reduced['manual_sentence_labels']):
    stratified_sample = sentences_val_reduced.iloc[sample_index]
    stratified_sample['manual_sentence_labels'] = stratified_sample['manual_sentence_labels'].replace(-1, np.nan)
    stratified_samples.append(stratified_sample)

# Determine the "examples" set with the highest count for each class
class_counts = pd.DataFrame()
for i, sample in enumerate(stratified_samples):
    counts = sample['manual_sentence_labels'].value_counts(dropna=False)
    counts.name = i
    class_counts = pd.concat([class_counts, counts], axis=1)

# Identify the sample that should be used as "examples"
examples_index = class_counts.idxmax(axis=1).value_counts().idxmax()

# Assign the selected sample as "examples"
examples = stratified_samples[examples_index]

# Assign the remaining samples to "fold1" through "fold5"
remaining_samples = [sample for i, sample in enumerate(stratified_samples) if i != examples_index]
folds_dict = {f"fold{i+1}": remaining_samples[i] for i in range(folds - 1)}

# Output the resulting folds
output_dfs = {"examples": examples}
output_dfs.update(folds_dict)

# Display the output
output_dfs["examples"], output_dfs["fold1"], output_dfs["fold2"]


In [ ]:
# Display the output
#output_dfs["examples"], output_dfs["fold1"], output_dfs["fold2"], output_dfs["fold3"], output_dfs["fold4"], output_dfs["fold5"],

In [ ]:
import pandas as pd

# List of fold names
fold_names = ["examples", "fold1", "fold2", "fold3", "fold4"]

# Add the 'fold' column to each dataframe
for name in fold_names:
    output_dfs[name]['fold'] = name

# Concatenate all dataframes into one
merged_df = pd.concat([output_dfs[name] for name in fold_names], ignore_index=True)

# Display the first 20 rows of the merged dataframe for verification
print(merged_df)



In [ ]:
merged_df['fold'] = merged_df['fold'].replace(['examples', 'fold1', 'fold2', 'fold3', 'fold4'], ['fold1', 'fold2', 'fold3', 'fold4', 'fold5'])


In [ ]:
print(merged_df['manual_sentence_labels'].value_counts(dropna=False))

In [ ]:
merged_df.to_csv('folds_test.csv', sep= '\t')

In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)




#print(merged_df[merged_df['relevance_manual'] == 0][['sentence', 'manual_sentence_labels']])
print(merged_df[merged_df['manual_sentence_labels'] == 3][['sentence', 'manual_sentence_labels']])